In [1]:
import nltk
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re

In [2]:
import statistics

In [98]:
latlib_df = pd.read_pickle("/Users/francesharrington/Documents/ling1340/Latin-Vocabulary-Analysis/data/latinlibrary_data.pkl")

In [99]:
ne_df = pd.read_pickle("/Users/francesharrington/Documents/ling1340/Latin-Vocabulary-Analysis/data/latinlibrary_ne_data.pkl")

In [5]:
#here's some of the data:
latlib_df.head()

,title,text,text_raw,tokens,token_count
0,Ammianus: Liber XXXI,AMMIANI MARCELLINI HISTORIAE LIBER XXXI ...,"[AMMIANI MARCELLINI HISTORIAE LIBER XXXI\n, \n...","[AMMIANI, MARCELLINI, HISTORIAE, LIBER, XXXI, ...",11030
1,Valerius Maximus I,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",10524
2,Valerius Maximus VIII,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",10337
3,Valerius Maximus IV,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",10852
4,Valerius Maximus VII,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",8918


In [6]:
ne_df.head()

,name,era
0,AMMIANVS MARCELLINVS,(4th century A.D.)
1,VALERIVS MAXIMVS,(1st century A.D.)
2,GAIVS VALERIVS FLACCVS,(died c. 90 A.D.)
4,P. VERGILIVS MARO,(70 – 19 B.C.)
6,ALBIVS TIBVLLVS,(c. 54 – 19 B.C.)


Here's some basic info on the data contained in latlib_df:

In [7]:
#overall shape of the dataframe:
latlib_df.shape

(672, 5)

In [8]:
latlib_df.describe()

,token_count
count,672.000000
mean,6331.989583
std,5564.159005
min,0.000000
25%,1681.500000
50%,5578.500000
75%,8590.500000
max,38414.000000


In [9]:
latlib_df.info()
#good! no null objects

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        672 non-null    object
 1   text         672 non-null    object
 2   text_raw     672 non-null    object
 3   tokens       672 non-null    object
 4   token_count  672 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 26.4+ KB


let's work on seperating works by style (prose or poetry):

In [10]:
#use Aeneid (P. VERGILI MARONIS AENEIDOS) + Ovid (P. OVIDI NASONIS) for verse examples

In [11]:
metamorphoses_ovid = latlib_df[latlib_df['text'].str.contains('P. OVIDI NASONIS METAMORPHOSEON')]

In [12]:
aeneid = latlib_df[latlib_df['text'].str.contains('P. VERGILI MARONIS AENEIDOS')]

In [13]:
aeneid

,title,text,text_raw,tokens,token_count
123,P. VERGILI MARONIS AENEIDOS LIBER DVODECIMVS,P. VERGILI MARONIS AENEIDOS LIBER DVODECIMVS ...,[P. VERGILI MARONIS AENEIDOS LIBER DVODECIMVS\...,"[P., VERGILI, MARONIS, AENEIDOS, LIBER, DVODEC...",7262
124,P. VERGILI MARONIS AENEIDOS LIBER VNDECIMVS,P. VERGILI MARONIS AENEIDOS LIBER VNDECIMVS O...,[P. VERGILI MARONIS AENEIDOS LIBER VNDECIMVS\n...,"[P., VERGILI, MARONIS, AENEIDOS, LIBER, VNDECI...",6884
125,P. VERGILI MARONIS AENEIDOS LIBER OCTAVVS,P. VERGILI MARONIS AENEIDOS LIBER OCTAVVS Ut ...,[P. VERGILI MARONIS AENEIDOS LIBER OCTAVVS\n\n...,"[P., VERGILI, MARONIS, AENEIDOS, LIBER, OCTAVV...",5420
126,Aeneid IV,P. VERGILI MARONIS AENEIDOS LIBER QVARTVS At ...,"[P. VERGILI MARONIS AENEIDOS LIBER QVARTVS\n, ...","[P., VERGILI, MARONIS, AENEIDOS, LIBER, QVARTV...",5408
127,P. VERGILI MARONIS AENEIDOS LIBER DECIMVS,P. VERGILI MARONIS AENEIDOS LIBER DECIMVS Pan...,"[P. VERGILI MARONIS AENEIDOS LIBER DECIMVS\n, ...","[P., VERGILI, MARONIS, AENEIDOS, LIBER, DECIMV...",6985
128,P. VERGILI MARONIS AENEIDOS LIBER SEPTIMVS,P. VERGILI MARONIS AENEIDOS LIBER SEPTIMVS Tu...,"[P. VERGILI MARONIS AENEIDOS LIBER SEPTIMVS\n,...","[P., VERGILI, MARONIS, AENEIDOS, LIBER, SEPTIM...",6058
129,P. VERGILI MARONIS AENEIDOS LIBER NONVS,P. VERGILI MARONIS AENEIDOS LIBER NONVS Atque...,"[P. VERGILI MARONIS AENEIDOS LIBER NONVS\n, \n...","[P., VERGILI, MARONIS, AENEIDOS, LIBER, NONVS,...",6222
130,P. VERGILI MARONIS AENEIDOS LIBER TERTIVS,P. VERGILI MARONIS AENEIDOS LIBER TERTIVS Pos...,"[P. VERGILI MARONIS AENEIDOS LIBER TERTIVS\n, ...","[P., VERGILI, MARONIS, AENEIDOS, LIBER, TERTIV...",5318
131,P. VERGILI MARONIS AENEIDOS LIBER SECVNDVS,P. VERGILI MARONIS AENEIDOS LIBER SECVNDVS Co...,"[P. VERGILI MARONIS AENEIDOS LIBER SECVNDVS\n,...","[P., VERGILI, MARONIS, AENEIDOS, LIBER, SECVND...",6069
132,P. VERGILI MARONIS AENEIDOS LIBER SEXTVS,P. VERGILI MARONIS AENEIDOS LIBER SEXTVS Sic ...,"[P. VERGILI MARONIS AENEIDOS LIBER SEXTVS\n, \...","[P., VERGILI, MARONIS, AENEIDOS, LIBER, SEXTVS...",6944


In [14]:
[x for x in aeneid.text_raw]
#I want raw text for this because there are line divisions--helpful for distinguishing verse from prose

[['P. VERGILI MARONIS AENEIDOS LIBER DVODECIMVS\n\n',
  '\n\n',
  '\nTurnus ut infractos adverso Marte Latinos',
  '\ndefecisse videt, sua nunc promissa reposci,',
  '\nse signari oculis, ultro implacabilis ardet',
  '\nattollitque animos. Poenorum qualis in arvis',
  '\nsaucius ille gravi venantum vulnere pectus\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0',
  '\ntum demum movet arma leo, gaudetque comantis',
  '\nexcutiens cervice toros fixumque latronis',
  '\nimpavidus frangit telum et fremit ore cruento:',
  '\nhaud secus accenso gliscit violentia Turno.',
  '\ntum sic adfatur regem atque ita turbidus infit:\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0',
  "\n'nulla mora in Turno; nihil est quod dicta retractent",
  '\nignavi Aeneadae, nec quae pepigere recusent:',
  '\ncongredior. fer sacra, pater, et concipe foedus.',
  '\naut hac Dardanium dextra sub Tartara mittam',
  '\ndesertorem Asiae (sedeant spectentque Latini),\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa

In [15]:
[len(x) for x in aeneid.text_raw]
#this gets me how many lines per-work there are (good metric for differentiating epic?)

[964, 924, 741, 715, 920, 833, 825, 727, 811, 905, 767, 880]

In [16]:
len([line for line in aeneid.text_raw][0][3])
#basically I want iterate like this for each line and find the average length of each line overall

44

In [17]:
count = 0
for i in aeneid.text_raw:
    count += len(i)
print(count) #this is how many lines are in the aeneid in total

10012


In [18]:
def MaxLineLength(l):
    return(len(max(l, key=len)))

In [19]:
max(aeneid.text_raw.map(MaxLineLength))
#so the longest line is 87 characters long, that's my threshold for a verse line (depending on Ovid I guess)

87

In [20]:
max(metamorphoses_ovid.text_raw.map(MaxLineLength)) #yep, 87 it is!

81

now to actually use that number to quantify prose v. verse!

In [103]:
latlib_df['max_line_length'] = latlib_df.text_raw.map(MaxLineLength)

In [104]:
latlib_df.head(5)

,title,text,text_raw,tokens,token_count,author,max_line_length
0,Ammianus: Liber XXXI,AMMIANI MARCELLINI HISTORIAE LIBER XXXI ...,"[AMMIANI MARCELLINI HISTORIAE LIBER XXXI\n, \n...","[AMMIANI, MARCELLINI, HISTORIAE, LIBER, XXXI, ...",11030,None,774
1,Valerius Maximus I,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",10524,None,2080
2,Valerius Maximus VIII,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",10337,None,1308
3,Valerius Maximus IV,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",10852,None,2034
4,Valerius Maximus VII,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",8918,None,2141


In [105]:
latlib_df.drop(latlib_df.index[latlib_df['token_count']<=10], inplace=True)


In [106]:
def Poems(l):
    if l <= 87:
        return('verse')
    else:
        return('prose')


In [107]:
latlib_df['style'] = latlib_df.max_line_length.map(Poems)

## sword word analysis:

In [27]:
#gladius:
gladius = latlib_df[(latlib_df.text.str.contains(pat = 'gladius', case=False)) | (latlib_df.text.str.contains(pat = 'gladii', case=False)) | (latlib_df.text.str.contains(pat = 'gladi', case=False)) | (latlib_df.text.str.contains(pat = 'gladio', case=False)) | (latlib_df.text.str.contains(pat = 'gladium', case=False)) | (latlib_df.text.str.contains(pat = 'gladie', case=False)) | (latlib_df.text.str.contains(pat = 'gladiorum', case=False)) | (latlib_df.text.str.contains(pat = 'gladiis', case=False)) | (latlib_df.text.str.contains(pat = 'gladios', case=False))]

In [28]:
#ferrum:
ferrum = latlib_df[(latlib_df.text.str.contains(pat = 'ferrum', case=False)) | (latlib_df.text.str.contains(pat = 'ferri', case=False)) | (latlib_df.text.str.contains(pat = 'ferro', case=False)) | (latlib_df.text.str.contains(pat = 'ferra', case=False)) | (latlib_df.text.str.contains(pat = 'ferrorum', case=False)) | (latlib_df.text.str.contains(pat = 'ferris', case=False))]

In [29]:
#mucro:
mucro = latlib_df[(latlib_df.text.str.contains(pat = 'mucro', case=False)) | (latlib_df.text.str.contains(pat = 'mucronis', case=False)) | (latlib_df.text.str.contains(pat = 'mucroni', case=False)) | (latlib_df.text.str.contains(pat = 'mucronem', case=False)) | (latlib_df.text.str.contains(pat = 'mucrone', case=False)) | (latlib_df.text.str.contains(pat = 'mucrones', case=False)) | (latlib_df.text.str.contains(pat = 'mucronum', case=False)) | (latlib_df.text.str.contains(pat = 'mucronibus', case=False))]

In [30]:
#ensis:
ensis = latlib_df[(latlib_df.text.str.contains(pat = 'ensis', case=False)) | (latlib_df.text.str.contains(pat = 'ensi', case=False)) | (latlib_df.text.str.contains(pat = 'ensem', case=False)) | (latlib_df.text.str.contains(pat = 'ense', case=False)) | (latlib_df.text.str.contains(pat = 'enses', case=False)) | (latlib_df.text.str.contains(pat = 'ensium', case=False)) | (latlib_df.text.str.contains(pat = 'ensibus', case=False))]

In [31]:
#capulus:
capulus = latlib_df[(latlib_df.text.str.contains(pat = 'capulus', case=False)) | (latlib_df.text.str.contains(pat = 'capuli', case=False)) | (latlib_df.text.str.contains(pat = 'capulo', case=False)) | (latlib_df.text.str.contains(pat = 'capulum', case=False)) | (latlib_df.text.str.contains(pat = 'capulorum', case=False)) | (latlib_df.text.str.contains(pat = 'capulis', case=False)) | (latlib_df.text.str.contains(pat = 'capulos', case=False))]

In [32]:
#let's see how many occurences of each word there are:
print('gladius: ',len(gladius))
print('ferrum: ',len(ferrum))
print('mucro: ',len(mucro))
print('ensis: ',len(ensis))
print('capulus: ',len(capulus)) #'capulus' is the least common, only appearing 48 times, let's look at tht data!

gladius:  277
ferrum:  462
mucro:  89
ensis:  573
capulus:  48


In [33]:
capulus.head()

,title,text,text_raw,tokens,token_count,max_line_length,style
1,Valerius Maximus I,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",10524,2080,prose
25,Statius: Achilleid II,PVBLIVS PAPINIVS STATIVS ACHILLEID II Exuit im...,"[PVBLIVS PAPINIVS STATIVS, \nACHILLEID II\n, \...","[PVBLIVS, PAPINIVS, STATIVS, ACHILLEID, II, Ex...",1243,71,verse
31,Statius: Thebaid XI,PVBLIVS PAPINIVS STATIVS THEBAID XI Postquam m...,"[PVBLIVS PAPINIVS STATIVS, \nTHEBAID XI\n, \nP...","[PVBLIVS, PAPINIVS, STATIVS, THEBAID, XI, Post...",6017,72,verse
34,Statius: Thebaid XII,PVBLIVS PAPINIVS STATIVS THEBAID XII Nondum cu...,"[PVBLIVS PAPINIVS STATIVS, \nTHEBAID XII\n, \n...","[PVBLIVS, PAPINIVS, STATIVS, THEBAID, XII, Non...",6244,72,verse
35,Statius: Thebaid IX,PVBLIVS PAPINIVS STATIVS THEBAID IX Asperat Ao...,"[PVBLIVS PAPINIVS STATIVS, \nTHEBAID IX\n, \nA...","[PVBLIVS, PAPINIVS, STATIVS, THEBAID, IX, Aspe...",6925,74,verse


Just from looking at these titles there's a pretty fair distribution of genre between the works (some philosophy, epic poetry, history, and even comedy plays!)

In [34]:
print(len(gladius[gladius['style']=='prose']))
print(len(gladius[gladius['style']=='verse']))
#the results here are skewed highly in favor of prose

221
56


In [35]:
print(len(ferrum[ferrum['style']=='prose']))
print(len(ferrum[ferrum['style']=='verse']))
#here there's more in the prose category

309
153


In [36]:
print(len(mucro[mucro['style']=='prose']))
print(len(mucro[mucro['style']=='verse']))
#it's closer here, but still in favor of prose

58
31


In [37]:
print(len(ensis[ensis['style']=='prose']))
print(len(ensis[ensis['style']=='verse']))
#heavily in favor of prose

406
167


In [38]:
print(len(capulus[capulus['style']=='prose']))
print(len(capulus[capulus['style']=='verse']))
#interestingly enough, verse is more popular than prose here

19
29


because the results are overwhelmingly in favor of prose in most categories let's look at how they match up in latlib_df

In [39]:
print(len(latlib_df[latlib_df['style']=='prose']))
print(len(latlib_df[latlib_df['style']=='verse']))
#so prose definitely shows more than verse, which is not super surprising

425
185


In [40]:
p_n = len(latlib_df[latlib_df['style']=='prose'])/len(latlib_df)*100
v_n = len(latlib_df[latlib_df['style']=='verse'])/len(latlib_df)*100
print('percentage of prose works in data: ',p_n,'%')
print('percentage of verse works in data: ',v_n,'%')

percentage of prose works in data:  69.67213114754098 %
percentage of verse works in data:  30.327868852459016 %


## below I'm going to add the aspects of ne_df to latlib_df for easier analysis:

In [110]:
ne_df['name'] = [w[:-2] for w in ne_df.name]

In [111]:
ne_df

,name,era
0,AMMIANVS MARCELLIN,(4th century A.D.)
1,VALERIVS MAXIM,(1st century A.D.)
2,GAIVS VALERIVS FLACC,(died c. 90 A.D.)
4,P. VERGILIVS MA,(70 – 19 B.C.)
6,ALBIVS TIBVLL,(c. 54 – 19 B.C.)
7,C. VELLEIVS PATERCVL,(c. 19 B.C. – A.D. 31)
8,M. TERENTIVS VAR,(116 – 27 B.C.)
9,P. TERENTIVS AF,(195/185 – c. 159 B.C.)
10,P. CORNELIVS TACIT,(c. 56 – c. 117 A.D.)
11,C. SVETONIVS TRANQVILL,(c. 69 – after 130)


In [151]:
def MatchName(l):
    for name in ne_df.name:
        last_word = name.split()[len(name.split()) - 1]
        if last_word in l:
            return name
            return "None"

In [154]:
latlib_df['author'] = latlib_df.text.map(MatchName)

to put the eras in the main df it's not going to be elegant, but I want to move on with my analysis so I'm going to merge the info into a new column for an easier method:

In [403]:
ne_df['merged'] = ne_df['era']+ne_df['name']

In [643]:
def MatchEra(l):
    for x in ne_df.merged:
        last_word = x.split()[len(x.split()) - 1]
        if last_word in l:
            clean = re.compile(r'\)\w* \w* \w*|\)\w* \w*|\)\w+\. \w* \w*|\)\w+|\(')
            return(clean.sub(' ', x))
            #return(x)
            #return "None"

In [644]:
latlib_df['era'] = latlib_df.text.map(MatchEra)

In [645]:
latlib_df.sample(10)

,title,text,text_raw,tokens,token_count,author,max_line_length,style,era
50,Persius,C. SVETONI TRANQVILLVI DE POETIS VITA AULI PER...,"[C. SVETONI TRANQVILLVI DE POETIS, VITA AULI P...","[C., SVETONI, TRANQVILLVI, DE, POETIS, VITA, A...",491,GAIVS VALERIVS FLACC,676,prose,died c. 90 A.D.
617,Cicero: Pro Rabiro Postumo,PRO C. RABIRIO POSTVMO ORATIO ...,"[PRO C. RABIRIO POSTVMO ORATIO\n, \n, \t, \t...","[PRO, C., RABIRIO, POSTVMO, ORATIO, I., Si, qu...",5134,None,969,prose,None
70,Tacitus: Annales XV,P. CORNELI TACITI ANNALIVM LIBER QVINTVS DECIM...,[P. CORNELI TACITI ANNALIVM LIBER QVINTVS DECI...,"[P., CORNELI, TACITI, ANNALIVM, LIBER, QVINTVS...",10255,P. CORNELIVS TACIT,1570,prose,c. 56 – c. 117 A.D.
365,Ovid: Heroides VI,P. OVIDI NASONIS EPISTVLAE HEROIDVM Lemnias H...,"[P. OVIDI NASONIS EPISTVLAE HEROIDVM\n, \n, \n...","[P., OVIDI, NASONIS, EPISTVLAE, HEROIDVM, Lemn...",1247,P. OVIDIVS NA,53,verse,43 B.C. – 17 A.D.
351,Ovid: Heroides XVII,P. OVIDI NASONIS EPISTVLAE HEROIDVM Si mihi ...,"[P. OVIDI NASONIS EPISTVLAE HEROIDVM\n, \n, \n...","[P., OVIDI, NASONIS, EPISTVLAE, HEROIDVM, Si, ...",2193,P. OVIDIVS NA,57,verse,43 B.C. – 17 A.D.
118,P. VERGILI MARONIS ECLOGA TERTIA,"P. VERGILI MARONIS ECLOGA TERTIA Dic mihi, ...","[P. VERGILI MARONIS ECLOGA TERTIA\n, \n\n, \n ...","[P., VERGILI, MARONIS, ECLOGA, TERTIA, Dic, mi...",979,P. VERGILIVS MA,66,verse,70 – 19 B.C.
491,Seneca the Elder,L. ANNAEI SENECAE MAIORIS CONTROVERSIARVM LIBE...,[L. ANNAEI SENECAE MAIORIS CONTROVERSIARVM LIB...,"[L., ANNAEI, SENECAE, MAIORIS, CONTROVERSIARVM...",22751,P. VERGILIVS MA,2250,prose,70 – 19 B.C.
565,Cicero: Academica,M. TVLLI CICERONIS ACADEMICI LIBRI QVATTVOR Li...,[M. TVLLI CICERONIS ACADEMICI LIBRI QVATTVOR\n...,"[M., TVLLI, CICERONIS, ACADEMICI, LIBRI, QVATT...",5576,M. TVLLIVS CICE,1984,prose,106 – 43 B.C.
267,Nepos: Life of Iphicrates,CORNELI NEPOTIS IPHICRATES [1] IPHICRATES Ath...,"[CORNELI NEPOTIS IPHICRATES\n\n, \n[1] IPHICR...","[CORNELI, NEPOTIS, IPHICRATES, [, 1, ], IPHICR...",497,CORNELIVS NEP,356,prose,c. 100 – 24 B.C.
89,"Varro: Lingua Latina, Fragmenta",M. TERENTI VARRONIS DE LINGUA LATINA FRAGMENTA...,"[M. TERENTI VARRONIS DE LINGUA LATINA\n, \nFRA...","[M., TERENTI, VARRONIS, DE, LINGUA, LATINA, FR...",2038,M. TERENTIVS VAR,988,prose,116 – 27 B.C.


In [150]:
#updated data samples so they reflect my current data:
examples = latlib_df.head(10)
examples.to_pickle("/Users/francesharrington/Documents/ling1340/Latin-Vocabulary-Analysis/data_samples/examples.pkl")